# 用代码下载MODIS数据

除了前面介绍的下载方法，如果你比较清楚你想下载的地方的modis tile code，直接使用一些开源代码会更加方便。modis 的tile code开以在[这里](https://modis-land.gsfc.nasa.gov/MODLAND_grid.html)人眼识别，也可以使用如下方法（参考：[Get Modis sinusoidal tile grid positions from latitude and longitude coordinates in Python](https://www.earthdatascience.org/tutorials/convert-modis-tile-to-lat-lon/)）。

In [1]:
import numpy as np

MODIS提供了一个文本文件，其中包含每个tile的经纬度坐标范围（我已经下载放到本文件夹下了，下载地址：https://modis-land.gsfc.nasa.gov/pdf/sn_bound_10deg.txt ）。把这些数据加载到一个numpy二维数组中。接下来，定义我们想要转换的点的坐标。

In [2]:
# first seven rows contain header information
# bottom 3 rows are not data
data = np.genfromtxt('sn_bound_10deg.txt', 
                     skip_header = 7, 
                     skip_footer = 3)
lat = 40.015
lon = -105.2705

现在我们可以循环浏览数组中的每一行（对应于一块tile），检查我们的点是否包含在该tile的坐标范围内。

In [3]:
in_tile = False
i = 0
while(not in_tile):
    in_tile = lat >= data[i, 4] and lat <= data[i, 5] and lon >= data[i, 2] and lon <= data[i, 3]
    i += 1

一旦到达一个包括你要搜索的点的tile，就退出循环，打印出tile的垂直和水平位置。

In [4]:
vert = data[i-1, 0]
horiz = data[i-1, 1]
print('Vertical Tile:', vert, 'Horizontal Tile:', horiz)

Vertical Tile: 4.0 Horizontal Tile: 9.0


使用此方法容易找到自己想要的点对应的tile。

接下来看看用代码快速下载modis数据的方法。参考资料：

- [jgomezdans/get_modis](https://github.com/jgomezdans/get_modis)

读取 home文件夹下 .netrc文件中的用户名和密码

In [6]:
import os
netrcDir = os.path.expanduser(os.path.join("~",".netrc"))

In [7]:
from netrc import netrc

In [8]:
urs = 'urs.earthdata.nasa.gov'
username=netrc(netrcDir).authenticators(urs)[0]
password=netrc(netrcDir).authenticators(urs)[2]

这里就不显示了，自己执行后就能看到username和password了，接着就在命令行中试一下下载数据。

在本文件夹下打开终端，并激活python环境，使用get_modis.py -h命令查看函数功能：

```Shell
conda activate aqualord
get_modis.py -h
```

-s 即平台必须是 MOLA (Aqua), MOLT (Terra) 或 MOTA (Combined) 之一。

想要确定自己想要的数据到底是什么平台的，可以在这个网站上查询：https://e4ftl01.cr.usgs.gov/

-p 产品必须在产品名称后面注明收集的内容，比如 MCD45A1.005

-b 和-e的标志是可选的，如果你只想得到完整的年份，可以忽略它们。

比如我们想要下载 mod13q1v006 数据，tile是 h27v06， 我们下载2020年的第一天（DoY为1）一直到第一个月最后一天（左闭右开区间，所以DoY为32），那么就执行下面的代码：

```Shell
get_modis.py -u <你的用户名> -P <你的密码> -v -p MOD13Q1.006 -s MOLT -y 2020 -t h27v06 -o data -b 1 -e 32
```

运行过程中会搜索需要下载的文件，比较慢，不是没反应，请耐心等待。

一次不要下载太多，否则会被服务器拒绝访问的，最好半年内。